# Stage 4 — Booking Core Feature Engineering
**Goal:** Create a single, clean feature table from the raw Booking.com data.

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType


BOOKING_CLEAN_PATH = "dbfs:/tmp/booking_clean/booking_clean.parquet"
OUTPUT_PATH = "dbfs:/tmp/booking_stage4/booking_features_complete_ml"

print(f"Loading data from {BOOKING_CLEAN_PATH}...")
df_raw = spark.read.parquet(BOOKING_CLEAN_PATH).dropDuplicates(["hotel_id"])

print(f"Total Hotels Loaded: {df_raw.count():,}")
display(df_raw.select("hotel_id", "property_highlights", "house_rules").limit(3))

Loading data from dbfs:/tmp/booking_clean/booking_clean.parquet...
Total Hotels Loaded: 3,239,391


hotel_id,property_highlights,house_rules
10000016,Free parking. Non-smoking rooms. 2 restaurants. Free Wifi. Family rooms. Free parking. Non-smoking rooms. 2 restaurants. Free Wifi. Family rooms,"List(List(Check-in, From 2:00 PM to 9:00 PM You need to let the property know what time you'll be arriving in advance.), List(Check-out, Until 11:00 AM), List(Cancellation/ prepayment, Cancellation and prepayment policies vary according to accommodation type. Enter your stay dates and check the conditions of your selected option.), List(Children & Beds, Child policies Children of all ages are welcome. Children 9 and above will be charged as adults at this property. To see correct prices and occupancy info, add the number and ages of children in your group to your search. Crib and extra bed policies The number of cribs allowed depends on the option you choose. Check your selected option for more info. There are no extra beds available at this property. All cribs are subject to availability.), List(No age restriction, There's no age requirement for check-in), List(Smoking, Smoking is not allowed.), List(Parties, Parties/events are not allowed), List(Quiet hours, Guests need be quiet between 10:00 PM and 8:00 AM.), List(Pets, Pets are allowed on request. Charges may apply.))"
10000172,Outdoor swimming pool. Free parking. Free Wifi. Non-smoking rooms. Outdoor swimming pool. Free parking. Free Wifi. Non-smoking rooms,"List(List(Check-in, From 2:00 PM to 6:00 PM You need to let the property know what time you'll be arriving in advance.), List(Check-out, Until 11:00 AM), List(Cancellation/ prepayment, Cancellation and prepayment policies vary according to accommodation type. Enter your stay dates and check the conditions of your selected option.), List(Children & Beds, Child policies Children of all ages are welcome. To see correct prices and occupancy info, add the number and ages of children in your group to your search. Crib and extra bed policies 0+ years Extra bed upon request Free The number of extra beds allowed depends on the option you choose. Check your selected option for more info. There are no cribs available at this property. All extra beds are subject to availability.), List(No age restriction, There's no age requirement for check-in), List(Payments by Booking.com, Booking.com takes your payment for this stay on behalf of the property, but make sure you have cash for any extras once you get there.), List(Smoking, Smoking is not allowed.), List(Parties, Parties/events are not allowed), List(Pets, Pets are not allowed.))"
10000278,Free parking. Free Wifi. Family rooms. Non-smoking rooms. Free parking. Free Wifi. Family rooms. Non-smoking rooms,"List(List(Check-in, From 2:00 PM to 11:00 PM You need to let the property know what time you'll be arriving in advance.), List(Check-out, Until 10:00 AM), List(Cancellation/ prepayment, Cancellation and prepayment policies vary according to accommodation type. Enter your stay dates and check the conditions of your selected option.), List(Children & Beds, Child policies Children of all ages are welcome. To see correct prices and occupancy info, add the number and ages of children in your group to your search. Crib and extra bed policies 0 - 2 years Crib upon request Free The number of cribs allowed depends on the option you choose. Check your selected option for more info. There are no extra beds available at this property. All cribs are subject to availability.), List(Age restriction, The minimum age for check-in is 18), List(Smoking, Smoking is not allowed.), List(Parties, Parties/events are not allowed), List(Quiet hours, Guests need be quiet between 10:00 PM and 8:00 AM.), List(Pets, Pets are not allowed.))"


## Section A: Amenities & Facilities

In [0]:
# 1. Prepare Sources
highlights_norm = F.trim(F.lower(F.coalesce(F.col("property_highlights"), F.lit(""))))

# FIX: Convert Array to String FIRST, then Lowercase
fac_str = F.lower(F.concat_ws(",", F.col("most_popular_facilities")))

df_amenities = (
    df_raw
    # --- KITCHEN (Part 1: From Highlights) ---
    .withColumn("has_kitchen", highlights_norm.contains("kitchen").cast("int"))

    # --- BASIC AMENITIES ---
    .withColumn("has_free_wifi", 
                (highlights_norm.contains("free wifi") | fac_str.contains("free wifi")).cast("int"))
    .withColumn("has_free_parking", 
                (highlights_norm.contains("free parking") | fac_str.contains("free parking")).cast("int"))
    .withColumn("has_breakfast", 
                (highlights_norm.contains("breakfast") | fac_str.contains("breakfast")).cast("int"))
    .withColumn("has_air_conditioning", fac_str.contains("air conditioning").cast("int"))
    
    # --- LUXURY / LEISURE ---
    .withColumn("has_swimming_pool", fac_str.contains("pool").cast("int"))  
    .withColumn("has_spa", (fac_str.contains("spa") | fac_str.contains("wellness")).cast("int"))
    .withColumn("has_fitness_center", (fac_str.contains("fitness") | fac_str.contains("gym")).cast("int"))
    .withColumn("has_restaurant", fac_str.contains("restaurant").cast("int"))
    .withColumn("has_room_service", fac_str.contains("room service").cast("int"))
    .withColumn("has_24hr_front_desk", fac_str.contains("24-hour front desk").cast("int"))

    # --- COMFORT ---
    .withColumn("has_coffee_maker", fac_str.contains("tea/coffee maker").cast("int"))
    .withColumn("has_family_rooms", (highlights_norm.contains("family rooms") | fac_str.contains("family rooms")).cast("int"))

)

cols_amenities = [
    "hotel_id",
    "has_kitchen", "has_free_wifi", "has_free_parking", "has_breakfast", "has_air_conditioning",
    "has_swimming_pool", "has_spa", "has_fitness_center", "has_restaurant", 
    "has_room_service", "has_24hr_front_desk", "has_coffee_maker", "has_family_rooms"
]

df_part_a = df_amenities.select(*cols_amenities)
display(df_part_a.limit(5))

hotel_id,has_kitchen,has_free_wifi,has_free_parking,has_breakfast,has_air_conditioning,has_swimming_pool,has_spa,has_fitness_center,has_restaurant,has_room_service,has_24hr_front_desk,has_coffee_maker,has_family_rooms
10000016,0,1,1,0,0,0,0,0,1,0,0,0,1
10000043,0,1,1,0,0,1,0,0,0,0,0,0,1
10000172,0,1,1,0,0,1,0,0,0,0,0,0,0
10000273,0,1,1,0,0,0,0,0,0,0,0,0,1
10000278,0,1,1,0,0,0,0,0,0,0,0,0,1


## Section B: Capacity & Visuals
We extract numeric features from the arrays:
* **Images:** More images usually imply a more professional listing.
* **Availability:** We check `max_guests` to see if the hotel can actually fit a family (4+ people).
* **Languages:** Counts how many languages the manager speaks.

In [0]:
df_capacity = (
    df_raw
    # Count Languages
    .withColumn("manager_language_count", F.size(F.col("manager_language_spoken")))
    # Count Images (Visual Richness)
    .withColumn("images_count", F.size(F.col("images")))
    # Metro Access (Explicit Boolean)
    .withColumn("has_metro_access", F.when(F.col("metro_railway_access") == True, 1).otherwise(0))
    # Guest Capacity (Aggregated from 'availability' array)
    .withColumn("max_guests", F.expr("aggregate(availability, 0L, (acc, x) -> greatest(acc, x.max_guests))"))
    # Room Variety
    .withColumn("num_room_types", F.expr("size(array_distinct(transform(availability, x -> x.room_type)))"))
    # Explicit Family Capacity Flag (Guests >= 4)
    .withColumn("has_family_capacity", F.when(F.col("max_guests") >= 4, 1).otherwise(0))
)

cols_capacity = [
    "hotel_id", 
    "manager_language_count", "images_count", "has_metro_access", 
    "max_guests", "num_room_types", "has_family_capacity"
]

df_part_b = df_capacity.select(*cols_capacity)
display(df_part_b.limit(5))

hotel_id,manager_language_count,images_count,has_metro_access,max_guests,num_room_types,has_family_capacity
10000172,3,37,0,10,1,1
10000454,4,13,0,5,1,1
10000723,2,24,0,4,1,1
10002811,2,41,0,6,1,1
10005243,1,60,1,6,1,1


## Section C: House Rules (Advanced Parsing)
The `house_rules` column is a complex array of structs. We explode it to analyze each rule individually.
* **Goal:** Determine if **Pets**,  or **Parties** are allowed.
* **Logic:** We look for the word "allowed" vs "not allowed" in the description.

In [0]:
rules_exploded = (
    df_raw
    .select("hotel_id", F.explode_outer("house_rules").alias("r"))
    .select(
        "hotel_id", 
        F.lower(F.col("r.rule")).alias("rule_l"), 
        F.lower(F.col("r.description")).alias("desc_l")
    )
)

# 2. Helper Logic
is_allowed = (F.col("desc_l").contains("allowed")) & (~F.col("desc_l").contains("not allowed"))
is_not_allowed = F.col("desc_l").contains("not allowed")

# 3. Aggregate Rules
df_part_c = (
    rules_exploded.groupBy("hotel_id")
    .agg(
        # PETS
        F.max(F.when(F.col("rule_l") == "pets", 
              F.when(is_allowed, 1).when(is_not_allowed, 0)).otherwise(None)).alias("pets_allowed"),
        
        # PARTIES
        F.max(F.when(F.col("rule_l") == "parties", 
              F.when(is_allowed, 1).when(is_not_allowed, 0)).otherwise(None)).alias("parties_allowed"),
        
        # QUIET HOURS
        F.max(F.when(F.col("rule_l") == "quiet hours", 1).otherwise(0)).alias("quiet_hours_enforced"),
        
        # AGE RESTRICTION
        F.max(F.when(F.col("rule_l") == "age restriction", 1).otherwise(0)).alias("age_restricted"),
    )
)

display(df_part_c.limit(5))

hotel_id,pets_allowed,parties_allowed,quiet_hours_enforced,age_restricted
10000016,1,0,1,0
10000172,0,0,0,0
10000278,0,0,1,1
10000295,0,0,0,0
10000440,1,0,0,0


## Section D: Property Surroundings (Landmarks)
We count how many official landmarks are within **2.0 km**.

In [0]:
df_landmarks = (
    df_raw
    .select("hotel_id", F.explode_outer("property_surroundings").alias("s"))
    .filter(F.col("s.distance") <= 2.0)
    .groupBy("hotel_id")
    .agg(F.count("*").alias("landmark_count_2km"))
)

display(df_landmarks.limit(5))

hotel_id,landmark_count_2km
10000016,1
10000278,2
10000295,1
10000454,1
10000506,10


## Section E: Property Info Text (Kitchen Booster)

In [0]:
# Normalize text
df_info = df_raw.withColumn("pi_lower", F.lower(F.col("property_information")))

df_part_e = (
    df_info
    .select("hotel_id", "pi_lower")
    .withColumn("has_kitchen_text", F.col("pi_lower").contains("kitchen").cast("int"))
    .drop("pi_lower")
)

display(df_part_e.limit(5))

hotel_id,has_kitchen_text
10000016,null
10000172,null
10000278,1
10000295,1
10000440,null


## Section F: Final Assembly & Save
**Important:** We fill `Null` values in boolean columns (like `pets_allowed`) with `0`. If the data doesn't say "Yes", we assume "No" to keep the ML model safe.

In [0]:
df_final = (
    df_part_a
    .join(df_part_b, "hotel_id", "left")
    .join(df_part_c, "hotel_id", "left")
    .join(df_landmarks, "hotel_id", "left")
    .join(df_part_e, "hotel_id", "left")
)

# 1. CONSOLIDATE KITCHEN FLAGS
df_final = (
    df_final
    .withColumn("has_kitchen_final", 
                F.greatest(F.col("has_kitchen"), F.col("has_kitchen_text")))
    .drop("has_kitchen", "has_kitchen_text")
    .withColumnRenamed("has_kitchen_final", "has_kitchen")
)
print(f"Assembled df_final with {len(df_final.columns)} columns")
display(df_final.limit(5))

Assembled df_final with 25 columns


hotel_id,has_free_wifi,has_free_parking,has_breakfast,has_air_conditioning,has_swimming_pool,has_spa,has_fitness_center,has_restaurant,has_room_service,has_24hr_front_desk,has_coffee_maker,has_family_rooms,manager_language_count,images_count,has_metro_access,max_guests,num_room_types,has_family_capacity,pets_allowed,parties_allowed,quiet_hours_enforced,age_restricted,landmark_count_2km,has_kitchen
10000172,1,1,0,0,1,0,0,0,0,0,0,0,3,37,0,10,1,1,0,0,0,0,null,0
10000454,0,0,0,0,0,0,0,0,0,0,0,0,4,13,0,5,1,1,1,0,1,0,1,1
10000723,0,1,0,0,1,0,0,0,0,0,0,1,2,24,0,4,1,1,1,0,1,0,null,1
10002811,1,1,0,1,0,0,0,0,0,0,0,1,2,41,0,6,1,1,0,0,0,0,null,1
10005243,1,1,0,1,0,0,0,0,0,0,0,1,1,60,1,6,1,1,1,0,0,0,10,0


In [0]:
def column_null_report(df):
    total_rows = df.count()

    null_exprs = [
        F.sum(F.col(c).isNull().cast("int")).alias(c)
        for c in df.columns
    ]
    null_counts = df.agg(*null_exprs).first().asDict()

    rows = [(c, int(null_counts[c]), float(null_counts[c]) / total_rows) for c in df.columns]
    return spark.createDataFrame(rows, ["column", "null_count", "null_rate"]).orderBy(F.desc("null_rate"))

null_report = column_null_report(df_final)
display(null_report)

column,null_count,null_rate
landmark_count_2km,1698663,0.5243772672085586
parties_allowed,788562,0.24342908898617055
pets_allowed,22564,0.006965506788158639
has_breakfast,0,0.0
has_metro_access,0,0.0
quiet_hours_enforced,0,0.0
max_guests,0,0.0
has_room_service,0,0.0
age_restricted,0,0.0
has_fitness_center,0,0.0


In [0]:
df_final = df_final.drop("parties_allowed")

In [0]:
fill_zero_cols  = [
    "pets_allowed", "quiet_hours_enforced", "age_restricted", 
    "landmark_count_2km"
]
df_final = df_final.na.fill(0, subset=fill_zero_cols )

print(f"Final Schema: {len(df_final.columns)} columns")
print(f"Saving to {OUTPUT_PATH}...")

df_final.write.mode("overwrite").parquet(OUTPUT_PATH)

print("SUCCESS! Final Booking Feature Table Saved.")
display(df_final.limit(5))

Final Schema: 24 columns
Saving to dbfs:/tmp/booking_stage4/booking_features_complete_ml...
SUCCESS! Final Booking Feature Table Saved.


hotel_id,has_free_wifi,has_free_parking,has_breakfast,has_air_conditioning,has_swimming_pool,has_spa,has_fitness_center,has_restaurant,has_room_service,has_24hr_front_desk,has_coffee_maker,has_family_rooms,manager_language_count,images_count,has_metro_access,max_guests,num_room_types,has_family_capacity,pets_allowed,quiet_hours_enforced,age_restricted,landmark_count_2km,has_kitchen
10000172,1,1,0,0,1,0,0,0,0,0,0,0,3,37,0,10,1,1,0,0,0,0,0
10000454,0,0,0,0,0,0,0,0,0,0,0,0,4,13,0,5,1,1,1,1,0,1,1
10000723,0,1,0,0,1,0,0,0,0,0,0,1,2,24,0,4,1,1,1,1,0,0,1
10002811,1,1,0,1,0,0,0,0,0,0,0,1,2,41,0,6,1,1,0,0,0,0,1
10005243,1,1,0,1,0,0,0,0,0,0,0,1,1,60,1,6,1,1,1,0,0,10,0
